In [0]:
# STREAM PROCESSING: AWS KINESIS

In [0]:
# Get Access and Secret Keys

In [0]:
%run "/Users/rjhelan@outlook.com/access_keys"

In [0]:
# Get cleaning functions

In [0]:
%run "/Users/rjhelan@outlook.com/data_cleaning_tools"

In [0]:
# The above cell is for importing cleaning functions from "data_cleaning_tools"
# Import necessary libraries
from pyspark.sql import DataFrame
from pyspark.sql.streaming import StreamingQuery
from pyspark.sql.types import *

Out[63]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1701472259931),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1701472259931),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1701472259931)]

In [0]:
# Task 4: Read data from Kinesis streams in Databricks

In [0]:
# Function to create data streams using kinesis stream names
def create_data_streams(kinesis_stream_name, schema):
    df = spark \
    .readStream \
    .format('kinesis') \
    .option('streamName', kinesis_stream_name) \
    .option('initialPosition','earliest') \
    .option('region','us-east-1') \
    .option('awsAccessKey', ACCESS_KEY) \
    .option('awsSecretKey', SECRET_KEY) \
    .load()

    # Deserialize the data column of the dataframe to a string
    df = df.selectExpr("CAST(data AS STRING) jsonData")

    # Parse the JSON string into a structured format using a predefined schema
    df = df.select(from_json("jsonData", schema).alias("data")).select("data.*")

    return df

In [0]:
# Function to create schemas using predefined columns name in each data streams
def create_schema(field_names):
    fields = []
    for field_name in field_names:
        # Each field has a name and is string type
        field = StructField(field_name, StringType(), True)
        fields.append(field)
    
    return StructType(fields)

In [0]:
# Predefined column names in each data stream from kinesis
pin_fields = ["category", "description", "downloaded", "follower_count", "image_src", "index", "is_image_or_video", "poster_name", "save_location", "tag_list", "title", "unique_id"]
geo_fields = ["country", "ind", "latitude", "longitude", "timestamp"]
user_fields = ["age", "date_joined", "first_name", "ind", "last_name"]

# Create schemas
pin_schema = create_schema(pin_fields)
geo_schema = create_schema(geo_fields)
user_schema = create_schema(user_fields)

# Create variables with similar name as Data streams when created on AWS Console
pin_kinesis_name = 'streaming-0a3db223d459-pin'
geo_kinesis_name = 'streaming-0a3db223d459-geo'
user_kinesis_name = 'streaming-0a3db223d459-user'

In [0]:
# Create data streams
df_pin = create_data_streams(pin_kinesis_name, pin_schema)
df_geo = create_data_streams(geo_kinesis_name, geo_schema)
df_user = create_data_streams(user_kinesis_name, user_schema)

In [0]:
# Task 5: Transform Kinesis streams in Databricks

In [0]:
# Clean the streaming data in the same way you have previously cleaned the batch data.
df_pin_cleaned = clean_df_pin(df_pin)
df_geo_cleaned = clean_df_geo(df_geo)
df_user_cleaned = clean_df_user(df_user)

In [0]:
# Task 6: Write the streaming data to Delta Tables

In [0]:
def save_to_delta_table(df: DataFrame, table_name: str)-> StreamingQuery:
    """
    Writes a streaming DataFrame to a Delta table
    
    Args:
    df (DataFrame): The streaming DataFrame to write.
    table_name (str): The name of the Delta table to write to.
    
    Returns:
    StreamingQuery: The handle to the streaming query.
    """
    # Attempt to write the stream to the Delta table
    try:
        stream_query = df.writeStream \
            .format("delta") \
            .outputMode("append") \
            .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
            .table(table_name)
        return stream_query
    except Exception as e:
        print(f"An error occurred while writing the stream to Delta table {table_name}: {e}")

In [0]:
# Delta table names for each dataframes
pin_table_name =  "0a3db223d459_pin_table"
geo_table_name =  "0a3db223d459_geo_table"
user_table_name =  "0a3db223d459_user_table"

# Save to delta table
save_to_delta_table(df_pin_cleaned, pin_table_name)
save_to_delta_table(df_geo_cleaned, geo_table_name)
save_to_delta_table(df_user_cleaned, user_table_name)

Out[58]: <pyspark.sql.streaming.StreamingQuery at 0x7f17e83f4790>

In [0]:
# Run this cell to delete checkpoint before re-starting to save files in delta table
dbutils.fs.rm("/tmp/kinesis/_checkpoints/", True)

Out[271]: True